In [1]:
import numpy as np

database -  http://www.manythings.org/anki/

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense
import numpy as np

batch_size= 64  # for training
epochs = 100
latent_dim = 256   
num_samples =10000         # number of samples to train on
data_path = 'mar.txt'

# Getting the data

In [3]:
# Vectorise the data  
input_texts = []                   # all english sentances in an array
target_texts = []                  # all marathi sentances in an array
input_characters = set()           # gives all the all the english characters
target_characters = set()          #  gives all the all the marathi characters

with open(data_path,'r',encoding='utf-8') as f:
    lines = f.read().split('\n')
    
for line in lines[:min(num_samples,len(lines)-1)]:
    # we can see in the data set they are seperated by tab space('/t') and the third one is garbage
    input_text,target_text, _ = line.split('\t')
#     print(input_text)
    # /n is put only to let us know this the end of the sentances
    target_text= '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
                                            
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)                                    

In [4]:
print(input_texts[:5])
print(target_texts[:5])

['Go.', 'Run!', 'Run!', 'Run!', 'Run!']
['\tजा.\n', '\tपळ!\n', '\tधाव!\n', '\tपळा!\n', '\tधावा!\n']


In [5]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens =  len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


print('Number of samples: ',len(input_texts))
print('Number of unique input characters: ',num_encoder_tokens)
print('Number of unique input characters: ',num_decoder_tokens)
print('Max sequence length of inputs: ',max_encoder_seq_length)
print('Max sequence length of ouputs: ',max_decoder_seq_length)

Number of samples:  10000
Number of unique input characters:  70
Number of unique input characters:  85
Max sequence length of inputs:  19
Max sequence length of ouputs:  42


# One hot coding the characters 

In [6]:
# converting into dict

input_token_index = dict(
    [(char,i) for i ,char in enumerate(input_characters)])

target_token_index = dict(
    [(char,i) for i ,char in enumerate(target_characters)])

# target_token_index

In [7]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 "'": 4,
 ',': 5,
 '-': 6,
 '.': 7,
 '0': 8,
 '1': 9,
 '2': 10,
 '3': 11,
 '4': 12,
 '5': 13,
 '6': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69}

In [9]:
# creating 3d arrays filled with zeros 

# num_pairs(total no. of sentance), max_english_sentence_length, num_english_characters

encoder_input_data = np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')

In [10]:
encoder_input_data.shape

(10000, 19, 70)

In [11]:
encoder_input_data[1,2:,0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [12]:
# one hot encoding

# zips create a tupple of the 2 lists afor every iteration in enumerate
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,char in enumerate(input_text):
        # if char is in input token dict it is on hotted
        encoder_input_data[i,t,input_token_index[char]] = 1
    encoder_input_data[i,t+1:,input_token_index[' ']] = 1
    for t,char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i,t,target_token_index[char]] = 1
        if t>0:
            # decoder_target_data will be ahead by one timestamp and will not 
            #include the start character
            decoder_target_data[i,t-1,target_token_index[char]] = 1
    decoder_input_data[i,t+1:,target_token_index[' ']] = 1
    decoder_target_data[i,t:,target_token_index[' ']] = 1

In [13]:
encoder_input_data[0].shape

(19, 70)

# LSTM

### Encoder

In [14]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense


# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states coz no ouputs for encoder
encoder_states = [state_h, state_c]

In [15]:
# Set up the decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
# we only want ouput and not other things
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [16]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)


Epoch 1/100
125/125 [==============================] - 43s 310ms/step - loss: 1.6029 - accuracy: 0.6391 - val_loss: 1.6684 - val_accuracy: 0.5955
Epoch 2/100
125/125 [==============================] - 31s 245ms/step - loss: 1.1520 - accuracy: 0.7088 - val_loss: 1.1888 - val_accuracy: 0.6877
Epoch 3/100
125/125 [==============================] - 30s 239ms/step - loss: 0.9091 - accuracy: 0.7632 - val_loss: 1.0366 - val_accuracy: 0.7246
Epoch 4/100
125/125 [==============================] - 33s 261ms/step - loss: 0.7970 - accuracy: 0.7850 - val_loss: 0.9731 - val_accuracy: 0.7350
Epoch 5/100
125/125 [==============================] - 37s 297ms/step - loss: 0.7305 - accuracy: 0.7992 - val_loss: 0.8891 - val_accuracy: 0.7560
Epoch 6/100
125/125 [==============================] - 38s 304ms/step - loss: 0.6835 - accuracy: 0.8111 - val_loss: 0.8525 - val_accuracy: 0.7668
Epoch 7/100
125/125 [==============================] - 36s 286ms/step - loss: 0.6454 - accuracy: 0.8211 - val_loss: 0.8266 -

Epoch 57/100
125/125 [==============================] - 27s 216ms/step - loss: 0.0836 - accuracy: 0.9734 - val_loss: 1.0383 - val_accuracy: 0.8051
Epoch 58/100
125/125 [==============================] - 31s 250ms/step - loss: 0.0813 - accuracy: 0.9743 - val_loss: 1.0357 - val_accuracy: 0.8038
Epoch 59/100
125/125 [==============================] - 32s 258ms/step - loss: 0.0788 - accuracy: 0.9747 - val_loss: 1.0539 - val_accuracy: 0.8034
Epoch 60/100
125/125 [==============================] - 31s 251ms/step - loss: 0.0770 - accuracy: 0.9751 - val_loss: 1.0579 - val_accuracy: 0.8032
Epoch 61/100
125/125 [==============================] - 35s 278ms/step - loss: 0.0737 - accuracy: 0.9764 - val_loss: 1.0609 - val_accuracy: 0.8041
Epoch 62/100
125/125 [==============================] - 35s 282ms/step - loss: 0.0721 - accuracy: 0.9766 - val_loss: 1.0674 - val_accuracy: 0.8042
Epoch 63/100
125/125 [==============================] - 35s 280ms/step - loss: 0.0706 - accuracy: 0.9770 - val_loss: 1

# Interferance mode (Sampling)



1.    encode input and retrieve initial decoder state
2.    run one step of decoder with this initial state and a "start of sequence" token as target. Output will be the next target token.
3.    Repeat with the current target token and current states



In [17]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)


In [18]:
# reverse tokening

reverse_input_char_index = dict(
    (i,char) for char,i in input_token_index.items()
)
reverse_target_char_index = dict(
    (i,char) for char,i in target_token_index.items()
)

In [19]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


# Predicting

In [20]:
for seq_index in range(100,200):
    
    # take one sequence (part of the trainign set)
    # for tying out decodin
    input_seq = encoder_input_data[seq_index:seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentences: ',input_texts[seq_index])
    print('Decoded sentences: ',decoded_sentence)

-
Input sentences:  I'm fat.
Decoded sentences:  मी जाडी आहे.

-
Input sentences:  I'm ill.
Decoded sentences:  मी आजारी आहे.

-
Input sentences:  It's OK.
Decoded sentences:  ठीक आहे.

-
Input sentences:  It's me!
Decoded sentences:  मी आहे!

-
Input sentences:  It's me.
Decoded sentences:  मी आहे.

-
Input sentences:  Me, too.
Decoded sentences:  मी पण.

-
Input sentences:  Me, too.
Decoded sentences:  मी पण.

-
Input sentences:  Open up.
Decoded sentences:  उघडा.

-
Input sentences:  Open up.
Decoded sentences:  उघडा.

-
Input sentences:  Perfect!
Decoded sentences:  परिपूर्ण!

-
Input sentences:  Show me.
Decoded sentences:  मला दाखवा.

-
Input sentences:  Show me.
Decoded sentences:  मला दाखवा.

-
Input sentences:  Shut up!
Decoded sentences:  गप्प!

-
Input sentences:  Shut up!
Decoded sentences:  गप्प!

-
Input sentences:  Shut up!
Decoded sentences:  गप्प!

-
Input sentences:  Tell me.
Decoded sentences:  मला सांगा.

-
Input sentences:  Tell me.
Decoded sentences:  मला सांगा.

